In [ ]:
from pathlib import Path
from zipfile import ZipFile

In [ ]:
data_dir = Path("../data")
if not data_dir.exists():
    data_dir.mkdir()
    z_path = Path("../malware-detection.zip")
    ZipFile(file=z_path).extractall(path=data_dir, )
    print(True)

In [ ]:
list(Path("../").glob("*"))